In [1]:
# imports
import os
from pkg_resources import resource_filename
import os
import numpy as np
from astropy.io import fits

# ASTR 257
from astr257 import img_utils

from utils import *

%load_ext autoreload
%autoreload 2

In [2]:
data_folder = "data-2019-09-23-nickel-joey-vivian"

In [10]:
OVERWRITE = False

## Calibration masters

In [4]:
biases_combined, header_biases = crawl_and_median(data_folder, 'biases')

darks_combined, header_darks = crawl_and_median(data_folder, 'darks')

flats_combined, header_flats = crawl_and_median(data_folder, 'flats')

In [6]:
darks_master = darks_combined - biases_combined
flats_master = flats_combined - darks_master - biases_combined

# Normalize the flat frame
flats_master_normalized = flats_master / np.median(flats_master)

# Save the masters
try:
    fits.writeto(os.path.join(data_folder, 'masters/darks_master.fits'), darks_master, header=header_darks, overwrite=OVERWRITE)
    fits.writeto(os.path.join(data_folder, 'masters/biases_combined.fits'), biases_combined, header=header_biases, overwrite=OVERWRITE)
    fits.writeto(os.path.join(data_folder, 'masters/flats_master_normalized.fits'), flats_master_normalized, header=header_flats, overwrite=OVERWRITE)
except:
    print('Warning: Did not overwrite file.')

## Science masters

In [7]:
pluto_combined, header_pluto = crawl_and_median(data_folder, 'pluto')

In [8]:
foo_factor = 1.1 * np.abs(np.min(flats_master_normalized))

In [9]:
pluto_master_full_cal = pluto_combined - biases_combined - darks_master / (flats_master_normalized + foo_factor)
try:
    fits.writeto(os.path.join(data_folder, 'masters/pluto_master_full_cal.fits'), pluto_master_full_cal, header=header_pluto, overwrite=OVERWRITE)
except:
    print('Warning: Did not overwrite file.')